In [1]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import json

In [2]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def get_current_weather(location, unit='imperial'):
    api_key = 'a07249621b3a9bf9b5015b69c219ea13'  # Replace with your OpenWeatherMap API key
    base_url = 'http://api.openweathermap.org/data/2.5/weather?'

    # Prepare the URL
    complete_url = f"{base_url}appid={api_key}&q={location}&units={unit}"

    # Make a request to the API
    response = requests.get(complete_url)
    data = response.json()

    if data['cod'] != 404:
        main = data['main']
        temperature = main['temp']
        return f"{temperature}°C" if unit == 'metric' else f"{temperature}°F"
    else:
        return "Weather data not found."

# Example usage
print(get_current_weather('Miami', 'imperial'))

90.37°F


In [4]:
# Create assistant and run it

user_message = "What is the weather in Miami?"
# user_message = "What is the weather in London?"

assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4o",
  tools=[{
      "type": "function",
      "function": {
          "name": "get_current_weather",
          "description": "Get the weather in a location",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string", "description": "City name"},
                  "unit": {"type": "string", "enum": ["metric", "imperial"]}
              },
              "required": ["location"]
          }
      }
    }
  ]
)


thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": user_message
    }
  ]
)


run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)


In [6]:
# Required action ID
# Wait a second of run isn't complete yet
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
required_action_id_1 = run.required_action.submit_tool_outputs.tool_calls[0].id

In [7]:
# Submitting functions outputs
location_from_input = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)["location"]
weather_output = get_current_weather(location_from_input, unit='imperial')

run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
      {
        "tool_call_id": required_action_id_1,
        "output": weather_output,
      },
    ]
)
run

Run(id='run_2cxISQBZGy7QACotXlkOuiVF', assistant_id='asst_fzvtJRguhZXzJ5k4aUeAXEOm', cancelled_at=None, completed_at=None, created_at=1719694257, expires_at=1719694857, failed_at=None, incomplete_details=None, instructions='You are a weather bot. Use the provided functions to answer questions.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1719694257, status='queued', thread_id='thread_f2QfewjvClK6e8AkI64Nw3Dy', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_current_weather', description='Get the weather in a location', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'City name'}, 'unit': {'type': 'string', 'enum': ['metric', 'imperial']}}, 'required': ['location']}), type='function')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), 

In [8]:
# Get and print full list of messages
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

user: What is the weather in Miami?
assistant: The current temperature in Miami is 90.37°F.
